In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
from PIL import Image


In [ ]:
torch.cuda.is_available()


In [ ]:
imsize = 256
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])
gpu_dtype = torch.cuda.FloatTensor


In [ ]:
def load_ResNet50_model(model_name):
    """load pre-trained model"""
    model = models.resnet50(pretrained=True, progress=True).type(gpu_dtype)
    model.load_state_dict(torch.load(model_name))
    return model

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU
​
def predict(model, image_name):
    """predict single image"""
    image = image_loader(image_name)
    model.eval()
    output = model(image)#.data.cpu().max(1)
    sm = torch.nn.Softmax(dim = 1)
    scores = sm(output)
    return torch.topk(scores, 2)
  
def prettify(tensor_topk):
    """transfer tensor object into dict with confidence level"""
    result = {}
    for i in range(tensor_topk.n_fields):
        result[tensor_topk.indices.data.cpu()[0][i].item()] = tensor_topk.values.data.cpu()[0][i].item()
    return result

In [ ]:
if __name__ == '__main__':
    model = load_ResNet50_model('test.ckpt')
    image = image_loader('1.jpg')
    # prettify result will be a dict -> {class: confidence_percentage}
    result = prettify(image) 
    